In [1]:
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import EmbeddingRetriever, TfidfRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

/home/ec2-user/anaconda3/envs/dev/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
document_store = ElasticsearchDocumentStore(
    host='3.83.13.28',
    username='',
    password='',
    index='bioasq',
    # similarity="cosine",
    # embedding_dim=256
)

In [4]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="dmis-lab/biobert-base-cased-v1.2",
)

In [5]:
document_store.update_embeddings(retriever)

Inferencing Samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [14:33<00:00, 18.59s/ Batches]
Updating embeddings: 10000 Docs [14:42, 11.33 Docs/s]                                                                                                                                                                       


In [30]:
reader = FARMReader(model_name_or_path="deepset/minilm-uncased-squad2", use_gpu=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
pipe = ExtractiveQAPipeline(reader, retriever)

In [35]:
prediction = pipe.run(
    query='What type of enzyme is peroxiredoxin 2 (PRDX2)?',
    params={"Retriever": {"top_k": 20}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.80 Batches/s]


In [36]:
from haystack.utils import print_answers

In [37]:
print_answers(prediction, details="minimum")


Query: What type of enzyme is peroxiredoxin 2 (PRDX2)?
Answers:
[   {   'answer': 'antioxidant',
        'context': 'Peroxiredoxin 2 (Prx2) is an antioxidant enzyme that uses '
                   'cysteine residues to decompose peroxides. Prx2 is the '
                   'third most abundant protein in erythro'},
    {   'answer': 'SGLT2',
        'context': 'Empagliflozin (Jardiance): a novel SGLT2 inhibitor for the '
                   'treatment of type-2 diabetes.'},
    {   'answer': 'Transcription factor',
        'context': 'Transcription factor forkhead box protein P2 (FOXP2) plays '
                   'an essential role in the development of language and '
                   'speech. However, the transcriptional a'},
    {   'answer': 'calpromotin',
        'context': 'ytic anemia associated with Heinz body formation. Prx2, '
                   'also known as calpromotin, regulates ion transport by '
                   'associating with the membrane and activa'},
    {   'ans

The first answer is correct!